In [ ]:
!pip install tpot


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.4/87.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 11.4 MB/s eta 0:00:00
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11939 sha256=c998adc548e4a6553c7cfdec1fae873638ea4d9b0a8dd4c5310ce6c912b2c9b0
  Stored in directory: /root/.cache/pip/wheels/af/f9/87/bf5b3d565c2a007b4dae9d8142dccc85a9f164e517062dd519
Successfully built stopit


In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Load data
train_data = pd.read_csv('Final_Training.csv')  # Adjust path
val_data = pd.read_csv('Final_val.csv')  # Adjust path

# Extract features and labels
X_train = train_data['Cleaned_Tweet']
y_train = train_data['Hate']  # You can choose 'Fake' as well

X_val = val_data['Cleaned_Tweet']
y_val = val_data['Hate']

# Convert text data into numerical format using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)  # Adjust number of features if needed
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)

# Convert sparse matrices to dense format
X_train_dense = X_train_tfidf.toarray()
X_val_dense = X_val_tfidf.toarray()

# Train-test split (if needed, you can skip this if already splitting correctly)
X_train, X_val, y_train, y_val = train_test_split(X_train_dense, y_train, test_size=0.2, random_state=42)

# Initialize TPOT
tpot = TPOTClassifier(verbosity=2, generations=5, population_size=20, random_state=42)

# Train the model
tpot.fit(X_train, y_train)

# Make predictions
y_pred = tpot.predict(X_val)

# Evaluate the model
accuracy = accuracy_score(y_val, y_pred)
print(f"Accuracy: {accuracy}")


Optimization Progress:   0%|          | 0/120 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.8465417000179943

Generation 2 - Current best internal CV score: 0.8489048975261071

Generation 3 - Current best internal CV score: 0.8489048975261071

Generation 4 - Current best internal CV score: 0.8489048975261071

Generation 5 - Current best internal CV score: 0.8489048975261071

Best pipeline: LogisticRegression(MultinomialNB(input_matrix, alpha=0.1, fit_prior=False), C=10.0, dual=False, penalty=l2)
Accuracy: 0.8714555765595463


In [ ]:
!pip install --upgrade scikit-learn scipy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 19.4 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.14.1 which is incompatible.


In [ ]:
import pickle
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Load data
train_data = pd.read_csv('Final_Training.csv')
val_data = pd.read_csv('Final_val.csv')

# Combine train and validation data for robust training
data = pd.concat([train_data, val_data], ignore_index=True)

# Ensure no missing values
data = data.dropna(subset=['Cleaned_Tweet', 'Hate', 'Fake'])  # Ensure all required columns are present

### Helper Function to Train and Save Model ###
def train_and_save_model(task_name, target_column):
    print(f"\nTraining model for: {task_name}")

    # Extract features and labels
    X = data['Cleaned_Tweet']
    y = data[target_column]

    # Check class imbalance and compute class weights
    class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y)
    class_weight_dict = {i: weight for i, weight in enumerate(class_weights)}
    print(f"Class Weights for {task_name}:", class_weight_dict)

    # Split data into train and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

    # Define the pipeline with TF-IDF and Logistic Regression
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(max_features=7000)),
        ('lr', LogisticRegression(class_weight='balanced', max_iter=1000))
    ])

    # Evaluate with StratifiedKFold
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = cross_val_score(pipeline, X_train, y_train, cv=kf, scoring='accuracy')
    print(f"Stratified K-Fold Cross-Validation Accuracy for {task_name}: {cv_scores.mean():.4f}")

    # Train the pipeline on the training data
    pipeline.fit(X_train, y_train)

    # Make predictions on the training and validation sets
    y_train_pred = pipeline.predict(X_train)
    y_val_pred = pipeline.predict(X_val)

    # Training Evaluation Metrics
    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_precision = precision_score(y_train, y_train_pred, average='binary')
    train_recall = recall_score(y_train, y_train_pred, average='binary')
    train_f1 = f1_score(y_train, y_train_pred, average='binary')
    train_macro_f1 = f1_score(y_train, y_train_pred, average='macro')
    print(f"\n{task_name} - Training Metrics:")
    print(f"Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1: {train_f1:.4f}, Macro F1: {train_macro_f1:.4f}")

    # Validation Evaluation Metrics
    val_accuracy = accuracy_score(y_val, y_val_pred)
    val_precision = precision_score(y_val, y_val_pred, average='binary')
    val_recall = recall_score(y_val, y_val_pred, average='binary')
    val_f1 = f1_score(y_val, y_val_pred, average='binary')
    val_macro_f1 = f1_score(y_val, y_val_pred, average='macro')
    print(f"\n{task_name} - Validation Metrics:")
    print(f"Accuracy: {val_accuracy:.4f}, Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, F1: {val_f1:.4f}, Macro F1: {val_macro_f1:.4f}")

    # Print classification report for validation set
    print(f"\n{task_name} - Classification Report (Validation Set):")
    print(classification_report(y_val, y_val_pred))

    # Save the trained pipeline
    model_filename = f"{task_name.lower()}_pipeline_model.pkl"
    with open(model_filename, 'wb') as f:
        pickle.dump(pipeline, f)
    print(f"{task_name} pipeline model saved successfully as {model_filename}")

# Train separate models for Hate and Fake
train_and_save_model("Hate", "Hate")
train_and_save_model("Fake", "Fake")



Training model for: Hate
Class Weights for Hate: {0: 0.9983277591973244, 1: 1.0016778523489933}
Stratified K-Fold Cross-Validation Accuracy for Hate: 0.8268

Hate - Training Metrics:
Accuracy: 0.9028, Precision: 0.8997, Recall: 0.9062, F1: 0.9030, Macro F1: 0.9028

Hate - Validation Metrics:
Accuracy: 0.8463, Precision: 0.8432, Recall: 0.8503, F1: 0.8468, Macro F1: 0.8463

Hate - Classification Report (Validation Set):
              precision    recall  f1-score   support

           0       0.85      0.84      0.85       837
           1       0.84      0.85      0.85       835

    accuracy                           0.85      1672
   macro avg       0.85      0.85      0.85      1672
weighted avg       0.85      0.85      0.85      1672

Hate pipeline model saved successfully as hate_pipeline_model.pkl

Training model for: Fake
Class Weights for Fake: {0: 0.9239442847667477, 1: 1.0897001303780964}
Stratified K-Fold Cross-Validation Accuracy for Fake: 0.8030

Fake - Training Metrics:

In [ ]:
import pickle
import pandas as pd

# Load test data
test_data = pd.read_excel('Test_Task_A.xlsx')  # Ensure 'test_data.csv' contains columns 'ID' and 'Cleaned_Tweet'

# Ensure no missing values in the required columns
test_data = test_data.dropna(subset=['Tweet'])

# Load the saved models
with open('hate_pipeline_model.pkl', 'rb') as f:
    hate_model = pickle.load(f)

with open('fake_pipeline_model.pkl', 'rb') as f:
    fake_model = pickle.load(f)

# Predict Hate labels (0/1)
test_data['Hate (0/1)'] = hate_model.predict(test_data['Tweet'])

# Predict Fake labels (0/1)
test_data['Fake (0/1)'] = fake_model.predict(test_data['Tweet'])

# Save the updated test data with predictions
output_filename = 'test_data_with_predictions.csv'
test_data.to_csv(output_filename, index=False)

print(f"Predictions added and saved to {output_filename}")


Predictions added and saved to test_data_with_predictions.csv
